In [1]:
import os
import sys
import svgutils.transform as sg

from jazzy import core, visualisation, helpers
# from cairosvg import svg2png

In [2]:
# Import config
opt_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'optimisation'))
sys.path.insert(0, opt_path)
import config

# Open the compressed data
data_path = os.path.abspath(os.path.join(os.getcwd(), '..', config.DATA_PATH))

In [4]:
# Read compound IDs and SMILES
robb_filepath = os.path.join(data_path, "hsd11b1_validation", "02-robb_data_smiles.txt")
with open(robb_filepath) as f: 
    data = f.readlines()
data = [entry.split("\t") for entry in data[1:]]
smiles_data = [(entry[1], entry[3].split("\n")[0]) for entry in data]
smiles_data

[('2', 'OC12CC3CC(C1)C(NC(=O)c4cnn5cccnc45)C(C3)C2'),
 ('3', 'OC12CC3CC(C1)C(NC(=O)c4cnc5cccnn45)C(C3)C2'),
 ('4', 'Cc1ccn2ncc(C(=O)NC3C4CC5CC3CC(O)(C5)C4)c2n1'),
 ('5', 'Cc1cnc2n(c1)ncc2C(=O)NC3C4CC5CC3CC(O)(C5)C4'),
 ('6', 'Cc1ccnc2n1ncc2C(=O)NC3C4CC5CC3CC(O)(C5)C4'),
 ('7', 'Cc1cc(C)n2ncc(C(=O)NC3C4CC5CC3CC(O)(C5)C4)c2n1'),
 ('8', 'COCc1cc(C)nc2n1ncc2C(=O)NC3C4CC5CC3CC(O)(C5)C4'),
 ('9', 'Cc1cc(C(F)F)n2ncc(C(=O)NC3C4CC5CC3CC(O)(C5)C4)c2n1'),
 ('10', 'Cc1cc(n2ncc(C(=O)NC3C4CC5CC3CC(O)(C5)C4)c2n1)C(F)(F)F'),
 ('11', 'OC12CC3CC(C1)C(NC(=O)c4cnn5c(ccnc45)C(F)(F)F)C(C3)C2'),
 ('12', 'OC12CC3CC(C1)C(NC(=O)c4cnn5c(cc(nc45)C6CC6)C(F)F)C(C3)C2'),
 ('13', 'COCc1cc(C)n2ncc(C(=O)NC3C4CC5CC3CC(O)(C5)C4)c2n1'),
 ('14', 'CCc1cnc2n(c1)ncc2C(=O)NC3C4CC5CC3CC(O)(C5)C4'),
 ('15', 'OC12CC3CC(C1)C(NC(=O)c4cnn5cc(Cl)cnc45)C(C3)C2'),
 ('16', 'OC12CC3CC(C1)C(NC(=O)c4cnn5cc(cnc45)C#N)C(C3)C2'),
 ('17', 'COc1cnc2n(c1)ncc2C(=O)NC3C4CC5CC3CC(O)(C5)C4'),
 ('18', 'OCCc1cnc2n(c1)ncc2C(=O)NC3C4CC5CC3CC(O)(C5)C4')]

In [5]:
def atomistic_strength_from_smiles_tuple(smiles_id_tuple):
    """Accepts a tuple with SMILES (element 1) and its ID (element 0) and generates an HB strength visualisation.
    """
    rdkit_mol = core.rdkit_molecule_from_smiles(smiles_id_tuple[1])
    kallisto_mol = core.kallisto_molecule_from_rdkit_molecule(rdkit_mol)
    atoms_and_nbrs = core.get_covalent_atom_idxs(rdkit_mol)
    charges = core.get_charges_from_kallisto_molecule(kallisto_mol, charge=0)
    atomic_map = core.calculate_polar_strength_map(rdkit_mol, kallisto_mol, atoms_and_nbrs, charges)
    img_text = visualisation.depict_strengths(rdkit_mol, 
                                            atomic_map, 
                                            fig_size=(450, 450),
                                            flatten_molecule=True, 
                                            highlight_atoms=True, 
                                            ignore_sdc=True, 
                                            ignore_sdx=True,
                                            ignore_sa=False,
                                            sdc_threshold=0.0, 
                                            sdx_threshold=0.0,
                                            sa_threshold=0.0)

    # Add ligand name as a title
    img_text = img_text.replace('svg:','')
    fig = sg.fromstring(img_text)
    label = sg.TextElement(250, 15, smiles_id_tuple[0], size=14, 
                        font='sans-serif', anchor='middle', color='#000000')
    fig.append(label)
    img_text = fig.to_str().decode("utf-8")
    return img_text

In [6]:
# Build strength visualisations
strengths = list()
for smiles_tuple in smiles_data:
    vis = atomistic_strength_from_smiles_tuple(smiles_tuple)
    strengths.append(vis)
    # output_filepath = smiles_tuple[0] + ".png"
    # svg2png(bytestring=vis, write_to=output_filepath)

In [7]:
class HorizontalDisplay:
    """
    Accepts a list of SVGs, concatenates them, and returns a horizonal rendering.
    """
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        concat_svgs = ''.join(self.args[0])
        template = '<div style="">{}</div>'
        return template.format(concat_svgs)

In [8]:
HorizontalDisplay(strengths)

In [9]:
def mol_vector_from_smiles_tuple(smiles_id_tuple):
    """Accepts a tuple with SMILES (element 1) and its ID (element 0) and generates a mol vector.
    """
    rdkit_mol = core.rdkit_molecule_from_smiles(smiles_id_tuple[1])
    kallisto_mol = core.kallisto_molecule_from_rdkit_molecule(rdkit_mol)
    atoms_and_nbrs = core.get_covalent_atom_idxs(rdkit_mol)
    charges = core.get_charges_from_kallisto_molecule(kallisto_mol, charge=0)
    atomic_map = core.calculate_polar_strength_map(rdkit_mol, kallisto_mol, atoms_and_nbrs, charges)
    mol_vector = helpers.sum_atomic_map(atomic_map)
    mol_vector["name"] = smiles_id_tuple[0]
    return mol_vector

In [10]:
# Build molecular vectors
vects = list()
for smiles_tuple in smiles_data:
    vects.append(mol_vector_from_smiles_tuple(smiles_tuple))
vects

[{'sdc': 8.222, 'sdx': 1.3172, 'sa': 4.3612, 'name': '2'},
 {'sdc': 8.0419, 'sdx': 1.2262, 'sa': 4.2025, 'name': '3'},
 {'sdc': 8.5052, 'sdx': 1.2498, 'sa': 4.3626, 'name': '4'},
 {'sdc': 8.6348, 'sdx': 1.2827, 'sa': 4.4039, 'name': '5'},
 {'sdc': 8.7113, 'sdx': 1.4513, 'sa': 4.5018, 'name': '6'},
 {'sdc': 9.0574, 'sdx': 1.4242, 'sa': 4.5582, 'name': '7'},
 {'sdc': 10.3905, 'sdx': 1.2197, 'sa': 5.2558, 'name': '8'},
 {'sdc': 9.5314, 'sdx': 1.4693, 'sa': 4.9929, 'name': '9'},
 {'sdc': 9.3376, 'sdx': 1.28, 'sa': 5.0441, 'name': '10'},
 {'sdc': 8.8229, 'sdx': 1.4541, 'sa': 5.0094, 'name': '11'},
 {'sdc': 10.1037, 'sdx': 1.2825, 'sa': 5.003, 'name': '12'},
 {'sdc': 10.1435, 'sdx': 1.2528, 'sa': 5.1433, 'name': '13'},
 {'sdc': 9.0078, 'sdx': 1.3243, 'sa': 4.476, 'name': '14'},
 {'sdc': 8.3519, 'sdx': 1.4923, 'sa': 4.604, 'name': '15'},
 {'sdc': 8.8175, 'sdx': 1.3023, 'sa': 4.8985, 'name': '16'},
 {'sdc': 9.2614, 'sdx': 1.273, 'sa': 5.0237, 'name': '17'},
 {'sdc': 9.4896, 'sdx': 2.0677, 'sa'